# 0) Versioning

V07: Update 2025 (due to old pandas code with depreciated methods):
- standard pd.json_normalize()-method instead of previous seperate pd.io.json
- replaced depreciated APPEND()-method with CONCAT()-method to build outpout dataframes
- Using openpyxl writer insted of depreciated xls file writer too

V05: Cycle-loop for JSON requests to RIS modified to retrieve ALL documents (BREAK only if empty data)

### Explanations on general data structure to be scraped:

We scrape info from the Ratsinformationsystem (RIS), more precisely the individual meetings of the different municipal organs, bodies and institutions.  
One page conatains the data of the 10 most recent meetings.  
The pages hold a nested information structure.  
Level1 contains meta-info on the meeting and links to some basic information  
Level2 has 2 interesting elements:  
(1) the links to "aux-files", aka the PDF-documents associated with the meeting and  
(2) the agendaItems: a detailed enumerations of subjects treated durcing the meeting  
  
As a result of the entire code, we want to produce 2 dataframes:  
(1) Containing a list with links to all "aux-files" plus the associated meta-info of the meeting  
(2) Containing all agenda items of the meeting, also with the associated meta-info included

# 1) Package Import

In [1]:
import pandas as pd
import requests
import json

In [2]:
# from pandas.io.json import json_normalize

In [3]:
import math

# 2) Variable declartations (global)

In [4]:
# number of pages scraped from the meeting-URL of the RIS
# (json request interface set to start with youngest entry and returning the data
#  of 10 meetings per page).... so e.g. 3 as value here is that the overall data
#  returned equals travelleing 30 meeting back in time from today)
##########################
v_requested_pages_no = 20
##########################
inc_p_no = 1
##########################

In [5]:
s_ris_url_meetings_1 = "http://ris-oparl.itk-rheinland.de/Oparl/bodies/0015/meetings?page=1"

In [6]:
##############################################################################################
s_ris_url_meetings_base = "http://ris-oparl.itk-rheinland.de/Oparl/bodies/0015/meetings?page="
##############################################################################################

# 3) Definitions

## 3a) Class definitions

## 3b) Function Definitions

# 4) Actual Program

## 4a) Data scraping from RIS: main loop reading the initial document pagees

In [7]:
# create empty list to accomodate ONE data (dictionaries) scraped from every RIS-page,
# not the meta-info (paging etc.)
# This list can be looped over after the scraping for further data manipulation

l_pages_data = ['void_1st_element']

In [8]:
while True:
    v_request_path = s_ris_url_meetings_base+str(inc_p_no)
    # print(v_request_path)

    # retrieve entire page content from RIS
    response = requests.get(v_request_path)

    # transform page info to dictionary
    d_response_json = response.json()
    
    # get only data payload as list from json response dictionary
    l_response_data = d_response_json.get('data')

################### ###########################################
    # if not l_response_data:    # BREAK loop if empty data payload >> no more pages
                                # 'not list' checks for empty list !!!!
                                # ALTERNATIVE: break for given value of inc_p_no >> control via no. of pages             
    if inc_p_no > 65:
        break
################################################################    
    else:
        ######################################################################
        # append to list ONLY the "data" payload of the page, represented as
        # one dictionary containing itself a nested dictionary structure
        # 
        # (the encoding of the pages payload data in the "data"-dictionary item
        # had to be identified via trial&error....)
        # https://jsoneditoronline.org was extremely helpful in this, allowing to 
        # visually explore the nested json structure of the original page
        #
        l_pages_data.append(l_response_data)
        ######################################################################
        
        inc_p_no += 1  # increment page counter so as to receive next page in next cycle loop

In [9]:
print(len(l_pages_data))
l_pages_data = l_pages_data[1:len(l_pages_data)] # eliminate first void "initialization"-list-element
print(len(l_pages_data))

66
65


In [10]:
len(l_pages_data[0]) # number of list elements for a given page

10

In [11]:
type(l_pages_data[2][9]) # type of a given list element in this nested structure

dict

In [12]:
# so now we have a list with one list element per scraped page
# and 10 dictionaries per list element, representing the payload data for 
# each of the 10 meetings per page (apparently a server default value)

In [13]:
# Example how to iterate over the result list and retrieve a specific dictionary value
# (in this case, the meeting-name)
# for p in range(v_requested_pages_no):
#     for m in range(10):
#        print(l_pages_data[p][m].get('name'))

## 4b) Create df for 1st info level for all scraped meeting pages: the meeting referentials

In [14]:
# initiate empty dataframe to accomodate the top level info meeting (level 1)
df_meetings = pd.DataFrame()

In [16]:
# DEPRECIATED APPEND method !
#for p in range(len(l_pages_data)):            # iterate over number of retrieved pages
#    for m in range(len(l_pages_data[p])):     # iterate over no. of meeting enties on given page
#        # print(l_pages_data[p][m].get('name'))
#        # Creating df by appending row for "json_normalized" row for every meeting on every page
#        df_meetings = df_meetings.append(pd.json_normalize(l_pages_data[p][m]), ignore_index = True, sort = False)

In [17]:
df_meetings = pd.concat([pd.json_normalize(meeting) 
                        for page in l_pages_data 
                        for meeting in page], 
                       ignore_index=True)

In [18]:
df_meetings.head(2)

,id,type,name,start,end,organization,auxiliaryFile,created,modified,location.id,...,resultsProtocol.id,resultsProtocol.type,resultsProtocol.name,resultsProtocol.fileName,resultsProtocol.mimeType,resultsProtocol.date,resultsProtocol.accessUrl,resultsProtocol.downloadUrl,resultsProtocol.created,resultsProtocol.modified
0,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,https://schema.oparl.org/1.1/Meeting,Rechnungsprüfungsausschuss,2025-01-28T15:00:00+01:00,2025-01-28T00:00:00+01:00,[http://ris-oparl.itk-rheinland.de/Oparl/bodie...,[{'id': 'http://ris-oparl.itk-rheinland.de/Opa...,2024-12-05T09:10:25+01:00,2025-01-20T05:54:17+01:00,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,https://schema.oparl.org/1.1/Meeting,Ordnungs- und Verkehrsausschuss,2025-01-15T16:00:00+01:00,2025-01-15T00:00:00+01:00,[http://ris-oparl.itk-rheinland.de/Oparl/bodie...,[{'id': 'http://ris-oparl.itk-rheinland.de/Opa...,2024-12-02T10:46:02+01:00,2025-01-16T12:05:02+01:00,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# df_meetings.columns.values # only needed when defining column name to be eliminated

In [20]:
# ...now starting to elimante the unneeded columns of this "meeting reference df":

# Define columns from df that are not needed in context
df_col_elim = (['agendaItem', 'auxiliaryFile', 'created', 'end', 'modified',
'participant','verbatimProtocol.accessUrl', 'verbatimProtocol.created',
'verbatimProtocol.fileName', 
'verbatimProtocol.mimeType', 'verbatimProtocol.modified',
'verbatimProtocol.type',
'resultsProtocol.accessUrl', 'resultsProtocol.created',
'resultsProtocol.fileName', 'resultsProtocol.type',
'resultsProtocol.mimeType', 'resultsProtocol.modified', 'resultsProtocol.date',
'resultsProtocol.id', 'resultsProtocol.name', 'resultsProtocol.downloadUrl'])

# Deduce list of columns that have to be maintained from this elemination list
l_col_keep = list(set(df_meetings.columns.values.tolist()) - set(df_col_elim))

# list can be arbitrary in sequence, so sorting fixes the issue and produces
# coherent output, with alphabetically sorted column names:
l_col_keep.sort()

# create data frames reduced to the columns that are really needed
df_meetings_2 = df_meetings[l_col_keep]


In [21]:
df_meetings_2.head(2)

,id,location.created,location.id,location.locality,location.modified,location.postalCode,location.room,location.streetAddress,location.subLocality,location.type,name,organization,start,type,verbatimProtocol.date,verbatimProtocol.downloadUrl,verbatimProtocol.id,verbatimProtocol.name
0,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,2019-03-26T08:42:23+01:00,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,Düsseldorf,2019-09-04T01:50:25+02:00,40213,Rathaus - Plenarsaal,Marktplatz 2,,https://schema.oparl.org/1.1/Location,Rechnungsprüfungsausschuss,[http://ris-oparl.itk-rheinland.de/Oparl/bodie...,2025-01-28T15:00:00+01:00,https://schema.oparl.org/1.1/Meeting,NaN,NaN,NaN,NaN
1,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,2019-03-26T08:42:23+01:00,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,Düsseldorf,2019-09-04T01:50:25+02:00,40213,Rathaus - Plenarsaal,Marktplatz 2,,https://schema.oparl.org/1.1/Location,Ordnungs- und Verkehrsausschuss,[http://ris-oparl.itk-rheinland.de/Oparl/bodie...,2025-01-15T16:00:00+01:00,https://schema.oparl.org/1.1/Meeting,2025-01-07,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,1. NachtragsTO


In [22]:
df_meetings_2.columns.values

array(['id', 'location.created', 'location.id', 'location.locality',
       'location.modified', 'location.postalCode', 'location.room',
       'location.streetAddress', 'location.subLocality', 'location.type',
       'name', 'organization', 'start', 'type', 'verbatimProtocol.date',
       'verbatimProtocol.downloadUrl', 'verbatimProtocol.id',
       'verbatimProtocol.name'], dtype=object)

In [31]:
l_pages_data[0][0]

{'id': 'http://ris-oparl.itk-rheinland.de/Oparl/bodies/0015/meetings/28023',
 'type': 'https://schema.oparl.org/1.1/Meeting',
 'name': 'Rechnungsprüfungsausschuss',
 'start': '2025-01-28T15:00:00+01:00',
 'end': '2025-01-28T00:00:00+01:00',
 'location': {'id': 'http://ris-oparl.itk-rheinland.de/Oparl/bodies/0015/locations/rn/200003',
  'type': 'https://schema.oparl.org/1.1/Location',
  'streetAddress': 'Marktplatz 2',
  'room': 'Rathaus - Plenarsaal',
  'postalCode': '40213',
  'subLocality': '',
  'locality': 'Düsseldorf',
  'created': '2019-03-26T08:42:23+01:00',
  'modified': '2019-09-04T01:50:25+02:00'},
 'organization': ['http://ris-oparl.itk-rheinland.de/Oparl/bodies/0015/organizations/gr/27'],
 'auxiliaryFile': [{'id': 'http://ris-oparl.itk-rheinland.de/Oparl/bodies/0015/files/541219',
   'type': 'https://schema.oparl.org/1.1/File',
   'name': 'Einladung',
   'fileName': '00541219.pdf',
   'mimeType': 'application/pdf',
   'date': '2025-01-14',
   'accessUrl': 'http://ris-oparl.

## 4c) Going for the 2nd level of info:  1. step: agenda items per meeting !

In [52]:
#len(l_pages_data[0][0].get('agendaItem'))

In [25]:
# initiate empty result df
df_meetings_agenda = pd.DataFrame()

In [26]:
for p in range(len(l_pages_data)):            # iterate over number of retrieved pages
    for m in range(len(l_pages_data[p])):     # iterate over no. of meeting enties on given page
        l_agenda_items = l_pages_data[p][m].get('agendaItem')   # get list with agenda item for given list
                                                                # list items are dictionaries
        if l_agenda_items == None:                              # check if there are agenda items at all !
            continue
        else:           
            df_meetings_agenda = pd.concat([df_meetings_agenda, pd.json_normalize(l_agenda_items)], 
                             ignore_index=True, sort=False)

In [27]:
print(df_meetings_agenda.columns.values)
print(len(df_meetings_agenda))
df_meetings_agenda.tail(2)

['id' 'type' 'meeting' 'number' 'order' 'name' 'public' 'created'
 'modified' 'consultation' 'result' 'auxiliaryFile']
18101


,id,type,meeting,number,order,name,public,created,modified,consultation,result,auxiliaryFile
18099,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,https://schema.oparl.org/1.1/AgendaItem,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,7.8,64,Pumptrack-Anlage auf dem Bolzplatz an der Quad...,True,2022-05-10T02:51:04+02:00,2024-12-30T11:07:34+01:00,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,zur Kenntnis genommen,NaN
18100,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,https://schema.oparl.org/1.1/AgendaItem,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,7.9,65,Nutzung Bürgerhaus Aloys-Odenthal-Platz - Info...,True,2022-05-13T02:06:00+02:00,2024-12-30T11:07:34+01:00,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,zur Kenntnis genommen,NaN


## 4d) Going for the 2nd level of info:  2. step: auxiliary files per meeting !

In [32]:
# l_pages_data[0][5].get('auxiliaryFile')
# l_pages_data[2][9].get('id')   # code to retrieve MEETING-ID, which needs to be added to aux_file_info

In [33]:
# initiate empty result df
df_meetings_aux_files = pd.DataFrame()

In [35]:
for p in range(len(l_pages_data)):            # iterate over number of retrieved pages
    for m in range(len(l_pages_data[p])):     # iterate over no. of meeting enties on given page
        l_aux_files = l_pages_data[p][m].get('auxiliaryFile')   # get list with agenda item for given list
                                                                # list items are dictionaries
        if l_aux_files == None:                                 # check if there are agenda items at all !
            continue
        else:
            df_aux = pd.json_normalize(l_aux_files)
            # eliminate unneeded columns
            l_col_keep = ['id', 'name', 'date', 'downloadUrl']
            df_aux = df_aux[l_col_keep]
            
            # data frame still needs to be enhanced with meeting referential info
            df_aux['meeting_id'] = l_pages_data[p][m].get('id')
            df_aux['meeting_name'] = l_pages_data[p][m].get('name') 
            
            df_meetings_aux_files = pd.concat([df_meetings_aux_files, df_aux], 
                                ignore_index=True, sort=False)


In [36]:
print(df_meetings_aux_files.columns.values)
print(len(df_meetings_aux_files))
df_meetings_aux_files.tail(2)

['id' 'name' 'date' 'downloadUrl' 'meeting_id' 'meeting_name']
2185


,id,name,date,downloadUrl,meeting_id,meeting_name
2183,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,"2. Nachtrag, TO",2022-05-20,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,Bezirksvertretung 7
2184,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,Niederschrift Ö,2022-06-13,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,Bezirksvertretung 7


## 4e) Joining referential info to 2nd level info and prettifying output

In [37]:
# print('df_mmeetings_2 column names: ' + df_meetings_2.columns.values)
# print('df_meetings_agenda column names: ' + df_meetings_agenda.columns.values)
# print('df_meetings_aux_files column names: ' + df_meetings_aux_files.columns.values)

### 4e I) Merging & prettification for agenda information

In [38]:
df_agenda = pd.merge(df_meetings_agenda, df_meetings_2, left_on='meeting', right_on = 'id')

In [39]:
# display raw result of table join
print(df_agenda.columns.values)
df_agenda.head(2)

['id_x' 'type_x' 'meeting' 'number' 'order' 'name_x' 'public' 'created'
 'modified' 'consultation' 'result' 'auxiliaryFile' 'id_y'
 'location.created' 'location.id' 'location.locality' 'location.modified'
 'location.postalCode' 'location.room' 'location.streetAddress'
 'location.subLocality' 'location.type' 'name_y' 'organization' 'start'
 'type_y' 'verbatimProtocol.date' 'verbatimProtocol.downloadUrl'
 'verbatimProtocol.id' 'verbatimProtocol.name']


,id_x,type_x,meeting,number,order,name_x,public,created,modified,consultation,...,location.subLocality,location.type,name_y,organization,start,type_y,verbatimProtocol.date,verbatimProtocol.downloadUrl,verbatimProtocol.id,verbatimProtocol.name
0,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,https://schema.oparl.org/1.1/AgendaItem,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,0,1,Öffentlicher Teil,True,2024-12-02T01:49:41+01:00,2024-12-30T11:07:34+01:00,NaN,...,,https://schema.oparl.org/1.1/Location,Ordnungs- und Verkehrsausschuss,[http://ris-oparl.itk-rheinland.de/Oparl/bodie...,2025-01-15T16:00:00+01:00,https://schema.oparl.org/1.1/Meeting,2025-01-07,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,1. NachtragsTO
1,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,https://schema.oparl.org/1.1/AgendaItem,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,1,2,Feststellung der Beschlussfähigkeit,True,2024-12-02T01:49:41+01:00,2024-12-30T11:07:34+01:00,NaN,...,,https://schema.oparl.org/1.1/Location,Ordnungs- und Verkehrsausschuss,[http://ris-oparl.itk-rheinland.de/Oparl/bodie...,2025-01-15T16:00:00+01:00,https://schema.oparl.org/1.1/Meeting,2025-01-07,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,1. NachtragsTO


In [40]:
# print(df_agenda.columns.values)

In [41]:
l_col_keep = ['id_x', 'name_x', 'number',
'result', 'auxiliaryFile', 'id_y', 'name_y', 'organization', 'start', 
'verbatimProtocol.date', 'verbatimProtocol.downloadUrl',
'verbatimProtocol.name']

df_agenda = df_agenda[l_col_keep]

d_col_rename = {'id_x': 'agenda_id', 'name_x': 'agenda_name', 'number': 'agenda_no',
'id_y': 'meeting_id', 'name_y': 'meeting_name', 
'verbatimProtocol.date': 'meeting_protocol_date', 'verbatimProtocol.downloadUrl': 'meeting_protocol_donwloadurl',
'verbatimProtocol.name': 'meeting_protocol_name'}

df_agenda.rename(columns=d_col_rename, inplace=True)

In [42]:
# retrieving only the download url of auxiliary files associated with agenda points
# >> auxiliaryFile column to be replaced >> retain only downloadURL from the stored dictionary !
# df['new column name'] = df['column name'].apply
#                     (lambda x: 'value if condition is met' if x condition else 'value if condition is not met')

df_agenda['agenda_aux_file'] = df_agenda['auxiliaryFile'].apply(lambda x: x[0].get('downloadUrl', '') if str(x) != 'nan' else '')



In [43]:
del df_agenda['auxiliaryFile']

In [44]:
print(df_agenda.columns.values)
print(len(df_agenda))
df_agenda.tail(2)

['agenda_id' 'agenda_name' 'agenda_no' 'result' 'meeting_id'
 'meeting_name' 'organization' 'start' 'meeting_protocol_date'
 'meeting_protocol_donwloadurl' 'meeting_protocol_name' 'agenda_aux_file']
18101


,agenda_id,agenda_name,agenda_no,result,meeting_id,meeting_name,organization,start,meeting_protocol_date,meeting_protocol_donwloadurl,meeting_protocol_name,agenda_aux_file
18099,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,Pumptrack-Anlage auf dem Bolzplatz an der Quad...,7.8,zur Kenntnis genommen,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,Bezirksvertretung 7,[http://ris-oparl.itk-rheinland.de/Oparl/bodie...,2022-05-24T17:00:00+02:00,2022-05-11,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,öffentliches digitales Sitzungspaket 24.05.2022,
18100,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,Nutzung Bürgerhaus Aloys-Odenthal-Platz - Info...,7.9,zur Kenntnis genommen,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,Bezirksvertretung 7,[http://ris-oparl.itk-rheinland.de/Oparl/bodie...,2022-05-24T17:00:00+02:00,2022-05-11,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,öffentliches digitales Sitzungspaket 24.05.2022,


### 4e II) Merging & prettification for auxilliary file information

In [45]:
df_aux_files = pd.merge(df_meetings_aux_files, df_meetings_2, left_on='meeting_id', right_on = 'id')

In [46]:
# display raw result of table join
print(df_aux_files.columns.values)
df_aux_files.head(2)

['id_x' 'name_x' 'date' 'downloadUrl' 'meeting_id' 'meeting_name' 'id_y'
 'location.created' 'location.id' 'location.locality' 'location.modified'
 'location.postalCode' 'location.room' 'location.streetAddress'
 'location.subLocality' 'location.type' 'name_y' 'organization' 'start'
 'type' 'verbatimProtocol.date' 'verbatimProtocol.downloadUrl'
 'verbatimProtocol.id' 'verbatimProtocol.name']


,id_x,name_x,date,downloadUrl,meeting_id,meeting_name,id_y,location.created,location.id,location.locality,...,location.subLocality,location.type,name_y,organization,start,type,verbatimProtocol.date,verbatimProtocol.downloadUrl,verbatimProtocol.id,verbatimProtocol.name
0,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,Einladung,2025-01-14,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,Rechnungsprüfungsausschuss,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,2019-03-26T08:42:23+01:00,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,Düsseldorf,...,,https://schema.oparl.org/1.1/Location,Rechnungsprüfungsausschuss,[http://ris-oparl.itk-rheinland.de/Oparl/bodie...,2025-01-28T15:00:00+01:00,https://schema.oparl.org/1.1/Meeting,NaN,NaN,NaN,NaN
1,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,Einladung,2025-01-06,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,Ordnungs- und Verkehrsausschuss,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,2019-03-26T08:42:23+01:00,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,Düsseldorf,...,,https://schema.oparl.org/1.1/Location,Ordnungs- und Verkehrsausschuss,[http://ris-oparl.itk-rheinland.de/Oparl/bodie...,2025-01-15T16:00:00+01:00,https://schema.oparl.org/1.1/Meeting,2025-01-07,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,1. NachtragsTO


In [47]:
print(df_aux_files.columns.values)

['id_x' 'name_x' 'date' 'downloadUrl' 'meeting_id' 'meeting_name' 'id_y'
 'location.created' 'location.id' 'location.locality' 'location.modified'
 'location.postalCode' 'location.room' 'location.streetAddress'
 'location.subLocality' 'location.type' 'name_y' 'organization' 'start'
 'type' 'verbatimProtocol.date' 'verbatimProtocol.downloadUrl'
 'verbatimProtocol.id' 'verbatimProtocol.name']


In [48]:
l_col_keep = ['id_x', 'name_x', 'date', 'downloadUrl', 
'id_y', 'name_y', 'organization', 'start', 'verbatimProtocol.date',
'verbatimProtocol.downloadUrl', 'verbatimProtocol.name']

df_aux_files = df_aux_files[l_col_keep]

d_col_rename = {'id_x': 'aux_file_id', 'name_x': 'aux_file_name', 
'id_y': 'meeting_id', 'name_y': 'meeting_name', 
'verbatimProtocol.date': 'meeting_protocol_date', 'verbatimProtocol.downloadUrl': 'meeting_protocol_donwloadurl',
'verbatimProtocol.name': 'meeting_protocol_name'}

df_aux_files.rename(columns=d_col_rename, inplace=True)

In [49]:
print(df_aux_files.columns.values)
print(len(df_aux_files))
df_aux_files.tail(2)

['aux_file_id' 'aux_file_name' 'date' 'downloadUrl' 'meeting_id'
 'meeting_name' 'organization' 'start' 'meeting_protocol_date'
 'meeting_protocol_donwloadurl' 'meeting_protocol_name']
2185


,aux_file_id,aux_file_name,date,downloadUrl,meeting_id,meeting_name,organization,start,meeting_protocol_date,meeting_protocol_donwloadurl,meeting_protocol_name
2183,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,"2. Nachtrag, TO",2022-05-20,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,Bezirksvertretung 7,[http://ris-oparl.itk-rheinland.de/Oparl/bodie...,2022-05-24T17:00:00+02:00,2022-05-11,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,öffentliches digitales Sitzungspaket 24.05.2022
2184,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,Niederschrift Ö,2022-06-13,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,Bezirksvertretung 7,[http://ris-oparl.itk-rheinland.de/Oparl/bodie...,2022-05-24T17:00:00+02:00,2022-05-11,http://ris-oparl.itk-rheinland.de/Oparl/bodies...,öffentliches digitales Sitzungspaket 24.05.2022


In [51]:
# Create Excel file with multiple sheets using openpyxl engine
with pd.ExcelWriter('_RIS_meeting_info.xlsx', engine='openpyxl') as writer:
    df_meetings_2.to_excel(writer, sheet_name='meeting_referential', index=True)
    df_aux_files.to_excel(writer, sheet_name='aux_files', index=True)
    df_agenda.to_excel(writer, sheet_name='agenda', index=True)
    # The writer will automatically save when exiting the context manager